In [1]:
# Compares contents of multiple models
import cobra
import pandas as pd
import cobra
from collections import OrderedDict
from copy import deepcopy
import os,sys
model_root_path = '../'
sys.path.append(model_root_path+'pycore/')

from gsm_custom_functions import *

In [2]:
from cobra.flux_analysis import flux_variability_analysis
model2_file = "./input/iRhtoC.json"
model3 = cobra.io.load_json_model(model2_file)

In [3]:
# find rxns with upper or lower bounds not equal to 1000, 0, or -1000
rxns = []
for rxn in model3.reactions:
    if rxn.upper_bound not in [1000, 0, -1000] or rxn.lower_bound not in [1000, 0, -1000]:
        rxns.append(rxn.id)
print('Reactions with bounds =/= ±1000 or 0: '+str(rxns))

Reactions with bounds =/= ±1000 or 0: ['EX_glc__D_e', 'EX_btn_c', 'ATPM_c']


In [4]:
# fvagsm = model1copy
# fvagsm.objective = obj1
# fvalist = [entry in different_reactions_model2 if entry.id != obj1]
# print(model1copy.objective)
with model3 as m:
    obj = m.reactions.get_by_id('BIOMASS')
    # obj.bounds = (0,1)
    fvaresult = cobra.flux_analysis.flux_variability_analysis(m, fraction_of_optimum=0)
    # report only values with minima of -900 or lower or maxima of 900 or higher
    fvaresult_high = fvaresult[(fvaresult.minimum < -900) | (fvaresult.maximum > 900)]
    display(fvaresult)
    display(fvaresult_high)

# for rxn in different_reactions_model2:
#     if rxn.id != obj1:
#         checkrxn = fvagsm.reactions.get_by_id(rxn.id)
#         flux_variability_analysis(fvagsm, reaction_list=checkrxn, loopless=False)

,minimum,maximum
THFAT_m,0.000000,0.000000
FMNRx_c,0.000000,0.000000
FMNRx2_c,0.000000,0.000000
ADPh_c,0.000000,201.687140
ADPh_m,0.000000,187.983237
...,...,...
OAADC_c,0.000000,201.687140
Ktps_e,0.000000,0.226681
G5SADr_m,-18.461538,0.000000
GHMT2r_m,-26.666667,11.590909


,minimum,maximum
AKGCITta_m,0.0,1000.0
CITMALta_m,-1000.0,60.0
AKGMALta_m,0.0,1000.0


In [10]:
sorted_fvaresult = fvaresult.loc[fvaresult.maximum > 100].sort_values(by='maximum', key=lambda x: abs(x))
print(sorted_fvaresult)
sorted_fvaresult = fvaresult.loc[fvaresult.minimum < -100].sort_values(by='minimum', key=lambda x: abs(x))
print(sorted_fvaresult)


               minimum      maximum
FACOAE161_x        0.0   100.843570
compACYLCOA_l      0.0   100.843570
FACOAE161_c        0.0   100.843570
MANNANH_c          0.0   100.843570
GLCGSD_c           0.0   100.843570
...                ...          ...
GLUDxi_c           0.0   233.687140
Ht_c_v             0.0   403.374279
Ht_c_m             0.0   610.000000
AKGMALta_m         0.0  1000.000000
AKGCITta_m         0.0  1000.000000

[236 rows x 2 columns]
               minimum     maximum
COAt_c_l    -100.84357  100.843570
STCOAt_c_l  -100.84357   11.375760
TAGt_l_rm   -100.84357  100.843570
DOLPt_c_r   -100.84357    0.000000
DUDPt_c_n   -100.84357    0.000000
...                ...         ...
UREA_c      -231.68714   37.683240
MTHFD_c     -233.68714   20.000000
H2Ot_c_m    -240.00000    2.060436
MDH_c       -251.68714   90.593570
CITMALta_m -1000.00000   60.000000

[69 rows x 2 columns]


In [11]:
# Report reactions that hit their upper or lower bounds
hit_bounds_rxns = []
for rxn in fvaresult.index:
    if fvaresult.at[rxn, 'minimum'] == model3.reactions.get_by_id(rxn).lower_bound or fvaresult.at[rxn, 'maximum'] == model3.reactions.get_by_id(rxn).upper_bound:
        hit_bounds_rxns.append(rxn)
print('Reactions that hit their upper or lower bounds: ')
for rxn in hit_bounds_rxns:
    print("Reaction:", rxn)
    print("Upper Bound:", model3.reactions.get_by_id(rxn).upper_bound, "vs Maximum Flux:", fvaresult.at[rxn, 'maximum'])
    print("Lower Bound:", model3.reactions.get_by_id(rxn).lower_bound, "vs Minimum Flux:", fvaresult.at[rxn, 'minimum'])
    print()

Reactions that hit their upper or lower bounds: 
Reaction: AKGCITta_m
Upper Bound: 1000.0 vs Maximum Flux: 1000.0
Lower Bound: 0.0 vs Minimum Flux: 0.0

Reaction: AKGMALta_m
Upper Bound: 1000.0 vs Maximum Flux: 1000.0
Lower Bound: 0.0 vs Minimum Flux: 0.0



In [4]:
fvaresult.to_csv('fva-results.csv')